# Quick start with pinocchio

Within this script we are going to
1. create a simple URDF model by hands, and compute inertia parameters for the bodies using `trimesh`
2. Pinocchio + meshcat
3. and try command for model and data from **[Pinocchio Cheat Sheet](https://github.com/stack-of-tasks/pinocchio/blob/master/doc/pinocchio_cheat_sheet.pdf)**

# 1. URDF + trimesh

**[URDF](https://arxiv.org/pdf/2308.00514)**, Unified Robot Description Format is an XML format for representing a robot model. URDF was initially been designed ti be used in Robot Operating System (ROS) tools and Gazebo simulator, but now most of simulators uses that format.

The **[URDF](https://wiki.ros.org/urdf/XML)** model consists of links and joints motion; the default URDF can be used for open chains only, for the closed chains extra steps must be done. 

File `two_bodies.urdf` contains 3 links:
* `world` which acts as a fixed root
* `body_one` is connected with `world` via `fixed_joint`
* `body_two` is related to `world` via `floating_joint`

<b>body_one</b>
Let us pick box shape for the `body_one` and pick sizes as [.4, .2, .1] m. The sizes must be written in `geometry` container. 

In order to fill in data to the `inertial` container, we need to compute data using `trimesh`.

In [ ]:
import trimesh as tr

box_one = tr.primitives.Box(extents = [.4, .2, .1])
box_one.density = 7800 # ~steel
print(f'mass = {box_one.mass}') # [kg]
print(f'inertia = {box_one.moment_inertia}') # [kg * m**2]

Now you can fill in `mass` and `inertia` values.

<b>body_two</b>

For the second body let us choose a cylinder shape. 

In [ ]:
cylinder = tr.primitives.Cylinder(radius=.05, height=.15)
cylinder_mass = 5
cylinder.density = cylinder_mass / cylinder.volume
print(cylinder.moment_inertia)

We can also us apply coordinate transformation if it is needed. 

In [ ]:
rotation = tr.transformations.euler_matrix(1, 0, 0, axes='sxyz')
cylinder.apply_transform(rotation)
cylinder.show()
print(cylinder.moment_inertia)

Now you can fill in `mass` and `inertia` values for the second body.

# 2. Pinocchio + meshcat

In [ ]:
from os import path
import pinocchio as pin
from pinocchio.visualize import MeshcatVisualizer
import meshcat

In [95]:
MODEL_NAME = "two_bodies.urdf"
ROOT = path.abspath('')
FOLDER_PATH = path.join(ROOT, '')
MODEL_PATH = path.join(FOLDER_PATH, MODEL_NAME)

model, collision_model, visual_model = pin.buildModelsFromUrdf(MODEL_PATH, None, pin.JointModelFreeFlyer())
data = model.createData()

In [ ]:
viz = MeshcatVisualizer(model, collision_model, visual_model)
viz.viewer = meshcat.Visualizer(zmq_url="tcp://127.0.0.1:6000")
viz.clean()
# meshcat-server --open

# Load the robot in the viewer.
viz.loadViewerModel()

# Display a robot configuration.
q0 = pin.neutral(model)
viz.display(q0)
viz.displayVisuals(True)

In [ ]:
hasattr(viz.viewer, 'jupyter_cell') and viz.viewer.jupyter_cell()

# 3. Model

In [ ]:
model.name

In [ ]:
model.names.tolist()

In [ ]:
model.joints.tolist()

In [ ]:
model.jointPlacements.tolist()

In [ ]:
model.inertias.tolist()

In [ ]:
print(model.nq, model.nv)


# 4. Data

In [83]:
q = pin.neutral(model)
pin.framesForwardKinematics(model, data, q)

In [ ]:
data.joints.tolist()

In [ ]:
data.oMi.tolist()

In [ ]:
data.oMf.tolist()

In [ ]:
data.v.tolist()

In [ ]:
data.a.tolist()

In [ ]:
data.f.tolist()

In [ ]:
data.M.tolist()

In [ ]:
data.nle.tolist()

In [ ]:
data.hg

In [ ]:
data.Ag

In [ ]:
data.Ig

In [ ]:
LINK_NAME = "body_two"
LINK_ID = model.getBodyId(LINK_NAME)
q = pin.neutral(model)
pin.framesForwardKinematics(model, data, q)
iMd = data.oMf[LINK_ID]
print(iMd)

In [ ]:
M = pin.SE3.Random()
print(M)


In [ ]:
M.rotation

In [ ]:
M.translation

In [ ]:
print(pin.SE3(M.rotation, M.translation))

In [ ]:
print(M.inverse())

In [ ]:
print(M * M.inverse())

In [ ]:
print(M.action)

In [ ]:
print(M.homogeneous)